In [1]:
%run init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, 
                                                  fixed_length_right=100, 
                                                  remove_stop_words=False)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9365.13it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5163.33it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 3032613.85it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9550.46it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5185.71it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 95088.12it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 9513.19it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7efc21e7ce48>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7efc16b9ec88>,
 'vocab_size': 16674,
 'embedding_input_dim': 16674,
 'input_shapes': [(10,), (100,)]}

In [5]:
model = mz.models.ArcII()

# load `input_shapes` and `embedding_input_dim` (vocab_size)
model.params.update(preprocessor.context)

model.params['task'] = ranking_task
model.params['embedding_output_dim'] = 100
model.params['embedding_trainable'] = True
model.params['num_blocks'] = 2
model.params['kernel_1d_count'] = 32
model.params['kernel_1d_size'] = 3
model.params['kernel_2d_count'] = [64, 64]
model.params['kernel_2d_size'] = [3, 3]
model.params['pool_2d_size'] = [[3, 3], [3, 3]]
model.params['optimizer'] = 'adam'

model.build()
model.compile()

print(model.params)

model_class                   <class 'matchzoo.models.arcii.ArcII'>
input_shapes                  [(10,), (100,)]
task                          Ranking Task
optimizer                     adam
with_embedding                True
embedding_input_dim           16674
embedding_output_dim          100
embedding_trainable           True
num_blocks                    2
kernel_1d_count               32
kernel_1d_size                3
kernel_2d_count               [64, 64]
kernel_2d_size                [3, 3]
activation                    relu
pool_2d_size                  [[3, 3], [3, 3]]
padding                       same
dropout_rate                  0.0


In [6]:
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1667400     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 10, 32)       9632        embedding[0][0]                  
__________

In [7]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
test_x, test_y = test_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=test_x, y=test_y, batch_size=len(test_y))

In [10]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20
)
print('num batches:', len(train_generator))

num batches: 102


In [11]:
history = model.fit_generator(train_generator, 
                              epochs=30, 
                              callbacks=[evaluate], 
                              workers=30, 
                              use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 6s 61ms/step - loss: 0.6123
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6165843661519929 - normalized_discounted_cumulative_gain@5(0.0): 0.6639951229938149 - mean_average_precision(0.0): 0.6255013171310638
Epoch 2/30
102/102 [==============================] - 11s 107ms/step - loss: 0.3213
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5795022399236163 - normalized_discounted_cumulative_gain@5(0.0): 0.6400033764936961 - mean_average_precision(0.0): 0.5927358064245385
Epoch 3/30
102/102 [==============================] - 12s 116ms/step - loss: 0.1556
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5734366624818106 - normalized_discounted_cumulative_gain@5(0.0): 0.6320835504730066 - mean_average_precision(0.0): 0.5873376891991933
Epoch 4/30
102/102 [==============================] - 12s 121ms/step - loss: 0.0966
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6047140052395196 - norma

In [12]:
append_params_to_readme(model)